## Imports

In [14]:
import torch
import os
import json
import matplotlib.pyplot as plt

In [15]:
import matplotlib
matplotlib.rcParams['figure.facecolor'] = 'white'

In [16]:
%run ../iu_xray.py

In [17]:
REPORTS_DIR = os.path.join(DATASET_DIR, 'reports')

## Apply chexpert labeler to reports

In [ ]:
%run -n ../../eval_report_generation_chexpert_labeler.py

In [ ]:
fname = os.path.join(REPORTS_DIR, 'reports.clean.json')
with open(fname, 'r') as f:
    clean_reports = list(json.load(f).values())
len(clean_reports)

In [ ]:
reports_by_filename = {
    r['filename']: r['clean_text']
    for r in clean_reports
}
len(reports_by_filename)

In [ ]:
df_reports = pd.DataFrame(reports_by_filename, columns=['reports'])
print(len(df_reports))
df_reports.head()

In [ ]:
%%time

labels = _apply_labeler_to_column(df_reports, 'reports',
                                  fill_empty=-2, fill_uncertain=-1)
labels.shape

In [ ]:
df_reports = _concat_df_matrix(df_reports, labels)
df_reports.head()

## Apply MIRQI labeler to reports

In [ ]:
from collections import defaultdict

In [ ]:
%run -n ../../eval_report_generation_mirqi.py

In [ ]:
path = os.path.join(REPORTS_DIR, 'reports_with_chexpert_labels.csv')
df = pd.read_csv(path, index_col=0)
df = df[['Reports', 'filename']]
df.head()

In [ ]:
%%time

mirqi_df = _apply_mirqi_to_df(df, gt_col_name='Reports', gen_col_name='Reports')
mirqi_df.head()

In [ ]:
valoration_to_int = {
    'POSITIVE': 1,
    'NEGATIVE': 0,
    'UNCERTAIN': -1,
}

WRONG_LEN_ATTRIBUTES = defaultdict(list)

def expand_attributes(row):
    attributes = row['attributes-gt']
    attributes = [s.strip('()') for s in attributes.split(') (')]
    attributes = [s.split('|') for s in attributes]
    
    for tup in attributes:
        if len(tup) != 4:
            WRONG_LEN_ATTRIBUTES['len-not-4'].append(tup)
            continue
        text, label, value, additional = tup
        row[label] = valoration_to_int[value]
        
    return row

In [ ]:
mirqi_df = mirqi_df.apply(expand_attributes, axis=1)
mirqi_df.head()

In [ ]:
base_cols = ['filename', 'Reports', 'attributes-gt', 'attributes-gen', 'MIRQI-r', 'MIRQI-p', 'MIRQI-f']
columns = base_cols + [c for c in mirqi_df if c not in base_cols]
mirqi_df = mirqi_df[columns]
mirqi_df.head()

In [ ]:
mirqi_df.replace(np.nan, -2, inplace=True)
mirqi_df.head()

In [ ]:
path = os.path.join(REPORTS_DIR, 'reports_with_mirqi_labels.csv')
mirqi_df.to_csv(path)

## Split sentences

In [ ]:
%run ../../utils/nlp.py

In [ ]:
from collections import Counter, defaultdict

In [ ]:
reports_fname = os.path.join(REPORTS_DIR, 'reports.clean.v2.json')
with open(reports_fname, 'r') as f:
    reports_as_dict = json.load(f)
    reports = list(reports_as_dict.values())
len(reports_as_dict), len(reports)

### Count appearances

In [ ]:
sentence_counter = get_sentences_appearances(r['clean_text'] for r in reports)
len(sentence_counter)

#### Check not-so-common sentences

In [ ]:
l = list(sentence_counter.items())
l = sorted(l, key=lambda x: x[1], reverse=True)
l[:20]

#### Check short sentences

In [ ]:
sorted([
    (sentence, len(appearances))
    for sentence, appearances in sentence_counter.items()
], key=lambda x: len(x[0]))

In [ ]:
target_sentence = 'hand .'
appearances = sentence_counter[target_sentence]
appearances

In [ ]:
target_report = appearances[0]
[
    report
    for report in reports
    if report['filename'] == target_report
]

## Label sentences with chexpert labels

In [ ]:
%run -n ../../metrics/report_generation/chexpert.py
# %run -n ../../eval_report_generation_chexpert_labeler.py

In [ ]:
columns = ['sentences', 'appearances']
df_sentences = pd.DataFrame([
    (sentence, len(appearances))
    for sentence, appearances in sentence_counter.items()
], columns=columns)
print(len(df_sentences))
df_sentences.head()

In [ ]:
%%time

labels = apply_labeler_to_column(df_sentences, 'sentences',
                                 fill_empty=-2, fill_uncertain=-1)
labels.shape

In [ ]:
df_sentences = _concat_df_matrix(df_sentences, labels)
print(len(df_sentences))
df_sentences.head()

In [ ]:
fpath = os.path.join(REPORTS_DIR, 'sentences_with_chexpert_labels.csv')

In [ ]:
df_sentences.to_csv(fpath, index=False)

## Load sentences for later use

In [18]:
import pandas as pd

In [19]:
fpath = os.path.join(REPORTS_DIR, 'sentences_with_chexpert_labels.csv')

In [20]:
df_sentences = pd.read_csv(fpath)
df_sentences.head()

,sentences,appearances,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Lesion,Lung Opacity,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,the cardiac silhouette and mediastinum size ar...,6,1.0,0.0,0.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0
1,there is no pulmonary edema .,12,1.0,-2.0,-2.0,-2.0,-2.0,0.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0
2,there is no focal consolidation .,34,1.0,-2.0,-2.0,-2.0,-2.0,-2.0,0.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0
3,there are no xxxx of a pleural effusion .,3,1.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,0.0,-2.0,-2.0,-2.0
4,there is no evidence of pneumothorax .,23,1.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,0.0,-2.0,-2.0,-2.0,-2.0


In [21]:
len(df_sentences)

6439

## Count sentences' groups

### Count normal vs abnormal

In [ ]:
snt_normal = df_sentences[df_sentences['No Finding'] == 1]
snt_abnormal = df_sentences[df_sentences['No Finding'] == 0]
len(snt_normal), len(snt_abnormal)

In [ ]:
snt_normal['appearances'].sum()

In [ ]:
snt_abnormal['appearances'].sum()

### Number of abnormal sentences per report

In [ ]:
is_sentence_normal = dict()
for index, row in df_sentences.iterrows():
    sentence = row['sentences']
    is_normal = row['No Finding']
    is_sentence_normal[sentence] = is_normal
len(is_sentence_normal)

In [ ]:
res = []
for report in reports:
    number_of_abnormal = sum(
        1 - is_sentence_normal[sentence]
        for sentence in split_sentences(report['clean_text'])
    )
    res.append(number_of_abnormal)
len(res)

In [ ]:
plt.hist(res)

### Top-K most common sentences

In [ ]:
cols = ['sentences', 'appearances']
df = df_sentences[cols].sort_values('appearances', ascending=False).head(5)
df

### Plot sentences appearances distribution

In [ ]:
len(df_sentences)

In [ ]:
df_sentences.sort_values('appearances', ascending=False).head()

In [ ]:
appearances = list(df_sentences['appearances'])

In [ ]:
plt.hist(appearances, bins=30)
plt.yscale('log')

plt.title('Sentence appearances distribution')

plt.ylabel('Number of sentences')
plt.xlabel('Number of appearances')

In [ ]:
plt.bar(list(range(len(values))), values)

## Collect synonyms

In [ ]:
import json
from collections import Counter, defaultdict

In [ ]:
%run ../vocab/__init__.py

In [ ]:
SYNONYMS = load_synonyms('iu_xray')
len(SYNONYMS)

In [ ]:
SYNONYMS = {}

In [ ]:
FOR_LATER = set()

In [ ]:
SEEN_SENTENCES = set()
for representative, syns in SYNONYMS.items():
    SEEN_SENTENCES.add(representative)
    for s in syns:
        SEEN_SENTENCES.add(s)
len(SEEN_SENTENCES)

In [ ]:
def is_number(s):
    try:
        s = int(s)
        return True
    except:
        return False

In [ ]:
def process_sentences(sentences, skip_later=True):
    index_to_repr = {
        index: representative
        for index, representative in enumerate(SYNONYMS.keys())
    }
    
    def _print_reprs():
        print('-'*20)
        for index, representative in index_to_repr.items():
            print(f'{index} - {representative}')
    
    def _add_new(sentence):
        index_to_repr[len(SYNONYMS)] = sentence
        SYNONYMS[sentence] = []
        SEEN_SENTENCES.add(sentence)

    def _add_as_syn(sentence, option):
        option = int(option)
            
        if option not in index_to_repr:
            print(f'No synonym found for option={option}')
            raise
        representative = index_to_repr[option]

        if representative not in SYNONYMS:
            print(f'representative {representative} not in SYNS')
            # Internal error!
            raise

        SYNONYMS[representative].append(sentence)
        SEEN_SENTENCES.add(sentence)
        
    _print_reprs()
    
    sentence_idx = 0
    while sentence_idx < len(sentences):
        sentence = sentences[sentence_idx]
        sentence = clean_sentence(sentence)
        
        if sentence in SEEN_SENTENCES or (not skip_later and sentence in FOR_LATER):
            sentence_idx += 1
            continue
            
        option = input(f'"{sentence}" --> ')
        
        if is_number(option):
            _add_as_syn(sentence, option)
            sentence_idx += 1
        elif option == 'l': # later
            FOR_LATER.add(sentence)
            sentence_idx += 1
        elif ',' in option: # split and allocate
            added_new = False
            for suboption in option.split(','):
                suboption = suboption.strip()
                if suboption == 'n':
                    new_sentence = input('\t\tInput new sentence: ')
                    _add_new(new_sentence)
                    added_new = True
                else:
                    _add_as_syn(sentence, suboption)

            if added_new:
                _print_reprs()
                
            sentence_idx += 1
        elif option == 'n': # new
            _add_new(sentence)
            _print_reprs()
            sentence_idx += 1
        elif option == 'b':
            print('Breaking')
            break
        else:
            print(f'Option not recognized: {option}')

In [ ]:
some_sentences = df_sentences.groupby('Fracture')['sentences'].apply(list)
some_sentences = sorted(some_sentences[0], key=lambda x: len(x))
process_sentences(some_sentences)

In [ ]:
save_synonyms('iu_xray', SYNONYMS)

In [ ]:
o

## Inspect Max amounts

### Max amount of words in a sentence

In [ ]:
sorted([(len(s.split()), s) for s in sentence_counter], reverse=True)

 ### Max amount of words

In [ ]:
max(len(report['clean_text'].split()) for report in reports)

### Max amount of sentences

In [ ]:
max(len(split_sentences(report['clean_text'])) for report in reports)

## Sentences + organs

### Attempt 1: Use chexpert-labels

Delete this

In [ ]:
from collections import namedtuple

In [ ]:
OrganLabeler = namedtuple('OrganLabeler',
                          ['mentions_other', 'mentions_heart', 'mentions_lungs'])

#### Heart rules

In [ ]:
grouped_heart = df_sentences.groupby([
    'Enlarged Cardiomediastinum',
    'Cardiomegaly',
])['sentences'].apply(list)
grouped_heart.head()

In [ ]:
grouped_heart[(-2.0, 0)]

In [ ]:
heart_col1 = 'Enlarged Cardiomediastinum'
heart_col2 = 'Cardiomegaly'

def mentions_heart(sample):
    empty1 = sample[heart_col1] == -2
    empty2 = sample[heart_col2] == -2
    return int(not empty1 or not empty2)

In [ ]:
sample = df_sentences.iloc[30]
sample['sentences'], mentions_heart(sample)

#### Lungs rules

In [ ]:
from collections import defaultdict
import re

In [ ]:
lungs_cols = [
    'Lung Lesion',
    'Lung Opacity',
    'Edema',
    'Consolidation',
    'Pneumonia',
    'Atelectasis',
    'Pneumothorax',
    'Pleural Effusion',
    'Pleural Other',
]

In [ ]:
grouped_lungs = df_sentences.groupby(
    lambda x: any(y != -2 for y in df_sentences.loc[x, lungs_cols]),
)['sentences'].apply(list)
grouped_lungs.head()

In [ ]:
lungs_appear = grouped_lungs[True]
len(lungs_appear)

In [ ]:
PATTERN_BOTH = re.compile(r'both|bilateral')
PATTERN_RIGHT = re.compile('right')
PATTERN_LEFT = re.compile('left')

In [ ]:
by_lung = defaultdict(list)

for sentence in lungs_appear:
    both = PATTERN_BOTH.search(sentence)
    right = PATTERN_RIGHT.search(sentence)
    left = PATTERN_LEFT.search(sentence)
            
    if left and right and both: key = 'all'
    elif left and right: key = 'left-right'
    elif left and both: key = 'both-left'
    elif both and right: key = 'both-right'
    elif both: key = 'both'
    elif right: key = 'right'
    elif left: key = 'left'
    else: key = 'none'
    
    by_lung[key].append(sentence)

[(k, len(g)) for k, g in by_lung.items()]

In [ ]:
by_lung['left']

In [ ]:
def mentions_lungs(sample):
    all_empty = all(label == -2 for label in sample[lungs_cols])
    
    if all_empty:
        return 0, 0
    
    sentence = sample['sentences']
    if PATTERN_BOTH.search(sentence):
        return 1, 1
    
    left = PATTERN_LEFT.search(sentence)
    right = PATTERN_RIGHT.search(sentence)
    
    if not right and not left:
        # None found ("both", "right", "left")
        return 1, 1
    
    return int(bool(left)), int(bool(right))

In [ ]:
# s = 'there is a 1 cm nodular opacity in the right costophrenic xxxx , increased since comparison examination .'
# 3523

# s =  'there is focal airspace disease in the right lung base concerning for pneumonia or aspiration .'
# 4777

s = 'left basilar opacity compatible pleural effusion and atelectasis .'
# 6058
df_sentences[df_sentences['sentences'] == s]

In [ ]:
sample = df_sentences.iloc[6058]
sample['sentences'], mentions_lungs(sample)

#### Other rules (background, bones)

In [ ]:
other_cols = [
    'Fracture',
    'Support Devices',
]

In [ ]:
grouped_other = df_sentences.groupby(
    lambda x: any(y != -2 for y in df_sentences.loc[x, other_cols]),
)['sentences'].apply(list)
grouped_other.head()

In [ ]:
other_present = grouped_other[True]
other_absent = grouped_other[False]
len(other_present), len(other_absent)

In [ ]:
other_present

In [ ]:
def mentions_other(sample):
    all_empty = all(label == -2 for label in sample[other_cols])
    
    return int(not all_empty)

In [ ]:
s = 'no displaced rib fracture visualized .'
# 250
df_sentences[df_sentences['sentences'] == s]

In [ ]:
sample = df_sentences.iloc[250]
sample['sentences'], mentions_other(sample)

#### Gather chexpert-label-based OrganLabeler

In [ ]:
chexpert_organ_labeler = OrganLabeler(
    mentions_other=mentions_other,
    mentions_heart=mentions_heart,
    mentions_lungs=mentions_lungs,
)

### Attempt 2: Regex-based

In [32]:
%run ../common/sentences2organs.py
%run ../common/constants.py

In [24]:
sentences = list(df_sentences['sentences'])
len(sentences)

6439

In [26]:
organs, errors = find_organs_for_sentences(sentences)
len(organs), len(errors['all-empty'])

(6439, 1219)

####  Review empty ones

In [ ]:
len(errors['all-empty'])

In [ ]:
sample = df_sentences.loc[df_sentences['sentences'].isin(set(errors['all-empty']))]
len(sample)

In [ ]:
sample['appearances'].sum(), df_sentences['appearances'].sum()

In [ ]:
sample.sort_values('appearances', ascending=False).head(40)

### Save to file

In [ ]:
organ_names = [ORGAN_BACKGROUND, ORGAN_HEART, ORGAN_LEFT_LUNG, ORGAN_RIGHT_LUNG]
df_organs = pd.DataFrame(organs, columns=organ_names)
sentences_and_organs = pd.concat([df_sentences, df_organs], axis=1).head()

In [ ]:
columns = ['sentences'] + organ_names
sentences_and_organs = sentences_and_organs[columns]

In [ ]:
fpath = os.path.join(REPORTS_DIR, 'sentences_with_organs.csv')
sentences_and_organs.to_csv(fpath, index=False)